In [4]:
#Import important packages

import os
import typing
#from sklearn.gaussian_process.kernels import *
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from matplotlib import cm
import gpytorch
from matplotlib import rcParams
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from torch.utils.data import TensorDataset, DataLoader
import math

In [5]:
# Set `EXTENDED_EVALUATION` to `True` in order to visualize your predictions.
EXTENDED_EVALUATION = True
EVALUATION_GRID_POINTS = 300  # Number of grid points used in extended evaluation
EVALUATION_GRID_POINTS_3D = 50  # Number of points displayed in 3D during evaluation


# Cost function constants
COST_W_UNDERPREDICT = 25.0
COST_W_NORMAL = 1.0
COST_W_OVERPREDICT = 10.0

In [6]:
class Model(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(Model, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                ard_num_dims=train_x.shape[1]
            )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    

def make_predictions(self, test_features: np.ndarray) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Predict the pollution concentration for a given set of locations.
    :param test_features: Locations as a 2d NumPy float array of shape (NUM_SAMPLES, 2)
    :return:
        Tuple of three 1d NumPy float arrays, each of shape (NUM_SAMPLES,),
        containing your predictions, the GP posterior mean, and the GP posterior stddev (in that order)
    """

    # TODO: Use your GP to estimate the posterior mean and stddev for each location here
    gp_mean = np.zeros(test_features.shape[0], dtype=float)
    gp_std = np.zeros(test_features.shape[0], dtype=float)
    
    model.eval()
    likelihood.eval()
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        f_x = model(val_X)
        mean = f_x.mean
        f_x_lower, f_x_upper = f_x.confidence_region()
        y = likelihood(f_x)
        y_lower, y_upper = y.confidence_region()

    return predictions, gp_mean, gp_std

def fitting_model(model, train_GT: np.ndarray,train_features: np.ndarray):
    """
    Fit your model on the given training data.
    :param train_features: Training features as a 2d NumPy float array of shape (NUM_SAMPLES, 2)
    :param train_GT: Training pollution concentrations as a 1d NumPy float array of shape (NUM_SAMPLES,)
    """

    # TODO: Fit your model here
    train_X, val_X, train_Y, val_Y = train_test_split(train_features,train_GT, 
                                                      train_size=0.9,test_size=0.1,
                                                      random_state=0,shuffle=True)
    
    val_X = torch.Tensor(val_X)
    val_Y = torch.Tensor(val_Y).squeeze()
    
    train_tensor_x = torch.Tensor(train_X);
    train_tensor_y = torch.Tensor(train_Y);
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = Model(train_tensor_x, train_tensor_y, likelihood)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


In [7]:
def cost_function(ground_truth: np.ndarray, predictions: np.ndarray) -> float:
    """
    Calculates the cost of a set of predictions.

    :param ground_truth: Ground truth pollution levels as a 1d NumPy float array
    :param predictions: Predicted pollution levels as a 1d NumPy float array
    :return: Total cost of all predictions as a single float
    """
    assert ground_truth.ndim == 1 and predictions.ndim == 1 and ground_truth.shape == predictions.shape

    # Unweighted cost
    cost = (ground_truth - predictions) ** 2
    weights = np.ones_like(cost) * COST_W_NORMAL

    # Case i): underprediction
    mask_1 = predictions < ground_truth
    weights[mask_1] = COST_W_UNDERPREDICT

    # Case ii): significant overprediction
    mask_2 = (predictions >= 1.2*ground_truth)
    weights[mask_2] = COST_W_OVERPREDICT

    # Weigh the cost and return the average
    return np.mean(cost * weights)


In [8]:
def perform_extended_evaluation(model: Model, output_dir: str = '/results'):
    """
    Visualizes the predictions of a fitted model.
    :param model: Fitted model to be visualized
    :param output_dir: Directory in which the visualizations will be stored
    """
    print('Performing extended evaluation')
    fig = plt.figure(figsize=(30, 10))
    fig.suptitle('Extended visualization of task 1')

    # Visualize on a uniform grid over the entire coordinate system
    grid_lat, grid_lon = np.meshgrid(
        np.linspace(0, EVALUATION_GRID_POINTS - 1, num=EVALUATION_GRID_POINTS) / EVALUATION_GRID_POINTS,
        np.linspace(0, EVALUATION_GRID_POINTS - 1, num=EVALUATION_GRID_POINTS) / EVALUATION_GRID_POINTS,
    )
    visualization_xs = np.stack((grid_lon.flatten(), grid_lat.flatten()), axis=1)

    # Obtain predictions, means, and stddevs over the entire map
    predictions, gp_mean, gp_stddev = model.make_predictions(visualization_xs)
    predictions = np.reshape(predictions, (EVALUATION_GRID_POINTS, EVALUATION_GRID_POINTS))
    gp_mean = np.reshape(gp_mean, (EVALUATION_GRID_POINTS, EVALUATION_GRID_POINTS))
    gp_stddev = np.reshape(gp_stddev, (EVALUATION_GRID_POINTS, EVALUATION_GRID_POINTS))

    vmin, vmax = 0.0, 65.0
    vmax_stddev = 35.5

    # Plot the actual predictions
    ax_predictions = fig.add_subplot(1, 3, 1)
    predictions_plot = ax_predictions.imshow(predictions, vmin=vmin, vmax=vmax)
    ax_predictions.set_title('Predictions')
    fig.colorbar(predictions_plot)

    # Plot the raw GP predictions with their stddeviations
    ax_gp = fig.add_subplot(1, 3, 2, projection='3d')
    ax_gp.plot_surface(
        X=grid_lon,
        Y=grid_lat,
        Z=gp_mean,
        facecolors=cm.get_cmap()(gp_stddev / vmax_stddev),
        rcount=EVALUATION_GRID_POINTS_3D,
        ccount=EVALUATION_GRID_POINTS_3D,
        linewidth=0,
        antialiased=False
    )
    ax_gp.set_zlim(vmin, vmax)
    ax_gp.set_title('GP means, colors are GP stddev')

    # Plot the standard deviations
    ax_stddev = fig.add_subplot(1, 3, 3)
    stddev_plot = ax_stddev.imshow(gp_stddev, vmin=vmin, vmax=vmax_stddev)
    ax_stddev.set_title('GP estimated stddev')
    fig.colorbar(stddev_plot)

    # Save figure to pdf
    figure_path = os.path.join(output_dir, 'extended_evaluation.pdf')
    fig.savefig(figure_path)
    print(f'Saved extended evaluation to {figure_path}')

    plt.show()


In [27]:
def main():
    # Load the training dateset and test features
    train_features = np.loadtxt('train_x.csv', delimiter=',', skiprows=1)
    train_GT = np.loadtxt('train_y.csv', delimiter=',', skiprows=1)
    test_features = np.loadtxt('test_x.csv', delimiter=',', skiprows=1)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    
    train_X, val_X, train_Y, val_Y = train_test_split(train_features,train_GT, 
                                                      train_size=0.8,test_size=0.2,
                                                      random_state=0,shuffle=True)
    
    val_X = torch.Tensor(val_X)
    val_Y = torch.Tensor(val_Y).squeeze()
    
    train_tensor_x = torch.Tensor(train_X);
    train_tensor_y = torch.Tensor(train_Y);
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = Model(train_tensor_x, train_tensor_y, likelihood)
    #optimizer_1 = torch.optim.Adam(model.parameters(), lr=2)
    #optimizer = torch.optim.LBFGS(model.parameters(), lr=3.0, max_iter=30,history_size=5)
    optimizer = torch.optim.Rprop(model.parameters(), lr=1.0, etas=(0.5, 1.2), step_sizes=(1e-06, 50))
    scheduler_reduce_lr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                                     factor=0.95, patience=10, 
                                                                     min_lr=0.05, threshold=0.001,
                                                                     eps=1e-08, 
                                                                     verbose=True)
    
    
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    model.train()
    likelihood.train()
    # Fit the model
    print('Fitting model')
    
    global iteration
    
    iteration = 0
    
    global last_best_loss
    
    #optimizer = optimizer_3
    
    #def closure():
        # Zero gradients from previous iteration
    for iteration in range(30):
        optimizer.zero_grad()
        # Output from model
        output = model(train_tensor_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_tensor_y)
        last_best_loss = loss
        loss.backward()
        print('Loss: %.3f, LR: %.3f, Iteration: %d' , (loss.item(),optimizer.param_groups[0]["lr"], iteration))
        optimizer.step()
        scheduler_reduce_lr.step(loss)
        
    #optimizer_3 = optimizer
    #optimizer = optimizer_3
    
    #def closure():
        # Zero gradients from previous iteration
        #for iteration in range(50):
    #    global iteration
            #global last_best_loss
        #global optimizer
    #    iteration +=1
    #    optimizer.zero_grad()
    #    torch.autograd.set_detect_anomaly(True)
        # Output from model
    #    output = model(train_tensor_x)
        # Calc loss and backprop gradients
    #    loss = -mll(output, train_tensor_y)
    #    last_best_loss = loss
    #    loss.backward()
    #    print('Loss: %.3f, LR: %.3f' , (loss.item(),optimizer.param_groups[0]["lr"]))
    #    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=999999999999.0, norm_type='inf')
    #   scheduler_reduce_lr.step(loss)
    #    save_checkpoint(loss.item(),model,optimizer,iteration)
    #    return loss
    
    #optimizer.step(closure)
    

    cost = cost_function(val_Y.cpu().numpy(), mean.cpu().numpy())
    print("Cost of evaluation set: %.3f",cost)

    #model = Model(train_features,train_GT,likelihood)
    #fitting_model(model,train_GT,train_features)

    # Predict on the test features
    #print('Predicting on test features')
    #predictions, gp_mean, gp_std = model.make_predictions(test_features)
    
    #print(predictions)
    
    #cost_function(train_GT, predictions)

    #if EXTENDED_EVALUATION:
    #    perform_extended_evaluation(model, output_dir='.')


if __name__ == "__main__":
    main()


Fitting model
Loss: %.3f, LR: %.3f, Iteration: %d (171.55404663085938, 1.0, 0)
Loss: %.3f, LR: %.3f, Iteration: %d (76.1441421508789, 1.0, 1)
Loss: %.3f, LR: %.3f, Iteration: %d (23.51742172241211, 1.0, 2)
Loss: %.3f, LR: %.3f, Iteration: %d (5.7478203773498535, 1.0, 3)
Loss: %.3f, LR: %.3f, Iteration: %d (4.58603048324585, 1.0, 4)
Loss: %.3f, LR: %.3f, Iteration: %d (5.351701259613037, 1.0, 5)
Loss: %.3f, LR: %.3f, Iteration: %d (4.635592460632324, 1.0, 6)
Loss: %.3f, LR: %.3f, Iteration: %d (3.537815570831299, 1.0, 7)
Loss: %.3f, LR: %.3f, Iteration: %d (3.213038444519043, 1.0, 8)
Loss: %.3f, LR: %.3f, Iteration: %d (3.07391357421875, 1.0, 9)
Loss: %.3f, LR: %.3f, Iteration: %d (2.998246669769287, 1.0, 10)
Loss: %.3f, LR: %.3f, Iteration: %d (2.8920302391052246, 1.0, 11)
Loss: %.3f, LR: %.3f, Iteration: %d (2.7258596420288086, 1.0, 12)
Loss: %.3f, LR: %.3f, Iteration: %d (2.5426735877990723, 1.0, 13)
Loss: %.3f, LR: %.3f, Iteration: %d (2.3082046508789062, 1.0, 14)
Loss: %.3f, LR: %.